In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall nltk -y
!pip install texthero

Uninstalling nltk-3.2.5:
  Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 1.4MB 17.3MB/s 
     |████████████████████████████████| 245kB 48.2MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434675 sha256=e40d5e1ee88615e9043d1d35cb1211017076c2917ae372b893a4a0b7328bad8c
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [3]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from tqdm import tqdm
import time
import nltk

import texthero as hero
from texthero import preprocessing

INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


データの読み込み

In [4]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))
all_title_abstract.head()

,title,abstract,cites,clean_title,clean_abstract
0,Heavy-Flavour Production at HERA,We review the theoretical and experimental s...,NaN,heavy flavour production hera,review theoretical experimental status hea...
1,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
2,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
3,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
4,A Modified Mixed Domain Method for Modeling Ac...,"In this paper, phase correction and amplitud...",NaN,modified mixed domain method modeling acoust...,paper phase correction amplitude compensati...


In [5]:
file = os.path.join(INPUT_DIR, 'w2v_model_title.dill')
title_w2vmodel = dill.load(open(file,'rb'))

In [6]:
# ベクトル化
tfidf_vectorizer = TfidfVectorizer(min_df=20, lowercase=False, token_pattern="(?u)\\b\\w+\\b")
tfidf_vectorizer.fit(all_title_abstract['clean_title'].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=None,
                min_df=20, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [7]:
idf_dic = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer._tfidf.idf_))

In [8]:
for word in title_w2vmodel.wv.index2word:
    if not word in idf_dic.keys():
        idf_dic[word] = 0

In [9]:
tqdm.pandas()

def des_to_mean_vec(text):
    
    try:
        text = text.strip()
        text_ls = [s for s in text.split(' ') if '' != s]
        return np.mean([title_w2vmodel.wv[word] * idf_dic[word] for word in text_ls if word in title_w2vmodel.wv.index2word], axis=0)
    except:
        return np.nan

df = all_title_abstract['clean_title'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  0%|          | 932/910608 [00:00<08:15, 1836.17it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 910608/910608 [08:09<00:00, 1859.63it/s]


0    [-0.064300396, 0.16435176, -0.25994095, -0.454...
1    [0.29481676, -0.4099668, 0.43148938, -0.146934...
2    [0.12793872, -0.48700076, 0.11925773, 0.084819...
3    [-0.12149168, 0.10123364, 0.0069773593, 0.1135...
4    [0.06035094, -0.13405944, -0.06374721, 0.08785...
Name: clean_title, dtype: object

In [10]:
title_df = pd.DataFrame()
for num in tqdm(list(range(0, 920000, 10000))):
    title_df = pd.concat([title_df, 
                          df.iloc[num:num+10000].apply(pd.Series)])
title_df.columns = ['w2v_tfidf_title_' + str(col) for col in title_df.columns]
title_df.head()

100%|██████████| 92/92 [03:54<00:00,  2.55s/it]


,w2v_tfidf_title_0,w2v_tfidf_title_1,w2v_tfidf_title_2,w2v_tfidf_title_3,w2v_tfidf_title_4,w2v_tfidf_title_5,w2v_tfidf_title_6,w2v_tfidf_title_7,w2v_tfidf_title_8,w2v_tfidf_title_9,w2v_tfidf_title_10,w2v_tfidf_title_11,w2v_tfidf_title_12,w2v_tfidf_title_13,w2v_tfidf_title_14,w2v_tfidf_title_15,w2v_tfidf_title_16,w2v_tfidf_title_17,w2v_tfidf_title_18,w2v_tfidf_title_19,w2v_tfidf_title_20,w2v_tfidf_title_21,w2v_tfidf_title_22,w2v_tfidf_title_23,w2v_tfidf_title_24,w2v_tfidf_title_25,w2v_tfidf_title_26,w2v_tfidf_title_27,w2v_tfidf_title_28,w2v_tfidf_title_29,w2v_tfidf_title_30,w2v_tfidf_title_31,w2v_tfidf_title_32,w2v_tfidf_title_33,w2v_tfidf_title_34,w2v_tfidf_title_35,w2v_tfidf_title_36,w2v_tfidf_title_37,w2v_tfidf_title_38,w2v_tfidf_title_39,...,w2v_tfidf_title_160,w2v_tfidf_title_161,w2v_tfidf_title_162,w2v_tfidf_title_163,w2v_tfidf_title_164,w2v_tfidf_title_165,w2v_tfidf_title_166,w2v_tfidf_title_167,w2v_tfidf_title_168,w2v_tfidf_title_169,w2v_tfidf_title_170,w2v_tfidf_title_171,w2v_tfidf_title_172,w2v_tfidf_title_173,w2v_tfidf_title_174,w2v_tfidf_title_175,w2v_tfidf_title_176,w2v_tfidf_title_177,w2v_tfidf_title_178,w2v_tfidf_title_179,w2v_tfidf_title_180,w2v_tfidf_title_181,w2v_tfidf_title_182,w2v_tfidf_title_183,w2v_tfidf_title_184,w2v_tfidf_title_185,w2v_tfidf_title_186,w2v_tfidf_title_187,w2v_tfidf_title_188,w2v_tfidf_title_189,w2v_tfidf_title_190,w2v_tfidf_title_191,w2v_tfidf_title_192,w2v_tfidf_title_193,w2v_tfidf_title_194,w2v_tfidf_title_195,w2v_tfidf_title_196,w2v_tfidf_title_197,w2v_tfidf_title_198,w2v_tfidf_title_199
0,-0.064300,0.164352,-0.259941,-0.454623,0.146733,-0.197167,-0.548098,0.110780,-0.291655,-0.313677,0.179201,0.162596,-0.090176,-0.042646,0.210342,0.138298,-0.243081,-0.157020,0.337780,-0.252986,-0.027969,0.243408,-0.236884,-0.190449,0.470831,-0.167309,-0.139265,0.162329,0.103070,-0.223092,0.184979,-0.408788,0.005633,0.116296,-0.213794,-0.529358,0.783422,-0.303236,-0.079729,-0.151903,...,-0.050786,-0.101494,-0.456395,-0.010509,0.266828,-0.542604,0.174360,-0.367819,0.041830,0.225833,0.024270,-0.875111,0.308093,-0.282525,-0.548295,-0.408291,-0.247216,-0.149109,-0.459826,-0.286050,-0.288359,-0.468780,-0.072787,0.542998,0.156530,0.276536,-0.730224,0.237394,0.200980,-0.398877,0.327404,-0.124501,0.309779,0.028045,0.230373,-0.408018,-0.286401,-0.275508,0.429890,0.491089
1,0.294817,-0.409967,0.431489,-0.146934,-0.439637,-0.633542,-0.200920,0.147842,0.318495,-0.082254,-0.149991,0.175321,0.156488,0.621983,0.512664,0.405764,0.401226,-0.087995,-0.266555,0.465741,-0.063598,0.177341,-0.089356,-0.142477,0.559332,-0.083649,0.348822,-0.244722,0.608626,-0.354587,0.252790,-0.143346,0.275545,-0.753459,-0.108833,-0.061113,0.047917,0.375884,0.199556,0.445493,...,-0.365381,0.103600,-0.788929,-0.005914,0.011325,-0.063762,-0.190256,-0.062703,0.454386,0.752036,-0.021106,-0.646803,-0.183629,0.016887,-0.107401,0.062571,-0.081899,-0.615822,0.312268,-0.245912,0.071890,-0.502690,-0.091416,-0.503213,0.573982,0.231961,-0.355579,-0.034144,0.384309,0.092085,0.175447,-0.017177,0.429792,-0.030185,0.261499,0.244042,-0.226558,0.083915,0.159689,0.283308
2,0.127939,-0.487001,0.119258,0.084819,-0.410200,-0.122226,-0.132372,-0.018573,-0.025096,-0.317752,0.397696,-0.083375,0.094551,0.130693,0.212089,-0.174388,0.025300,0.135909,0.122248,0.129645,-0.237781,-0.091762,0.166251,-0.147300,0.197780,-0.192190,0.166307,0.456626,0.315516,-0.231767,-0.013137,-0.163171,-0.028506,-0.315183,-0.092975,-0.008731,0.474467,-0.026492,0.445037,-0.116543,...,-0.044082,-0.081781,-0.441576,0.056336,0.317809,-0.180145,-0.408403,-0.108533,0.267355,0.401045,0.263342,-0.567791,-0.271442,0.047748,-0.090655,-0.361392,0.454338,-0.123266,-0.407303,-0.242244,0.006947,-0.518553,0.062747,-0.002323,0.550753,0.513625,-0.532866,0.018385,0.108409,0.218584,-0.159615,-0.113470,0.839842,-0.048404,0.386550,0.083272,-0.003092,-0.273969,-0.145010,0.389702
3,-0.121492,0.101234,0.006977,0.113582,-0.292496,-0.492694,-0.388896,0.060518,0.173200,0.147834,-0.141208,-0.278006,-0.236649,-0.069968,

In [11]:
title_df.to_feather(os.path.join(INPUT_DIR, 'title_w2v_tfidf_df.feather'))

del title_df
gc.collect()

0

In [12]:
file = os.path.join(INPUT_DIR, 'w2v_model_abstract.dill')
abstract_w2vmodel = dill.load(open(file,'rb'))

In [13]:
# ベクトル化
tfidf_vectorizer = TfidfVectorizer(min_df=20, lowercase=False, token_pattern="(?u)\\b\\w+\\b")
tfidf_vectorizer.fit(all_title_abstract['clean_abstract'].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=None,
                min_df=20, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [14]:
idf_dic = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer._tfidf.idf_))

In [15]:
for word in abstract_w2vmodel.wv.index2word:
    if not word in idf_dic.keys():
        idf_dic[word] = 0

In [16]:
tqdm.pandas()

def des_to_mean_vec(text):
    
    try:
        text = text.strip()
        text_ls = [s for s in text.split(' ') if '' != s]
        return np.mean([abstract_w2vmodel.wv[word] * idf_dic[word] for word in text_ls if word in abstract_w2vmodel.wv.index2word], axis=0)
    except:
        return np.nan

df = all_title_abstract['clean_abstract'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  4%|▎         | 31975/910608 [06:38<2:57:58, 82.28it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 910608/910608 [3:09:24<00:00, 80.13it/s]


0    [0.013840052, -0.22822021, 0.020194905, -0.101...
1    [-0.2621447, -0.1981912, 0.19492723, -0.059471...
2    [-0.23886952, -0.04294739, 0.067668274, -0.078...
3    [-0.29642585, -0.18651317, 0.08695053, -0.0645...
4    [-0.14593555, -0.30127183, 0.13488719, 0.08857...
Name: clean_abstract, dtype: object

In [17]:
abstract_df = pd.DataFrame()
for num in tqdm(list(range(0, 920000, 10000))):
    abstract_df = pd.concat([abstract_df, 
                             df.iloc[num:num+10000].apply(pd.Series)])
# abstract_df = df.apply(pd.Series)
abstract_df.columns = ['w2v_tfidf_abstract_' + str(col) for col in abstract_df.columns]
abstract_df.head()

100%|██████████| 92/92 [04:07<00:00,  2.69s/it]


,w2v_tfidf_abstract_0,w2v_tfidf_abstract_1,w2v_tfidf_abstract_2,w2v_tfidf_abstract_3,w2v_tfidf_abstract_4,w2v_tfidf_abstract_5,w2v_tfidf_abstract_6,w2v_tfidf_abstract_7,w2v_tfidf_abstract_8,w2v_tfidf_abstract_9,w2v_tfidf_abstract_10,w2v_tfidf_abstract_11,w2v_tfidf_abstract_12,w2v_tfidf_abstract_13,w2v_tfidf_abstract_14,w2v_tfidf_abstract_15,w2v_tfidf_abstract_16,w2v_tfidf_abstract_17,w2v_tfidf_abstract_18,w2v_tfidf_abstract_19,w2v_tfidf_abstract_20,w2v_tfidf_abstract_21,w2v_tfidf_abstract_22,w2v_tfidf_abstract_23,w2v_tfidf_abstract_24,w2v_tfidf_abstract_25,w2v_tfidf_abstract_26,w2v_tfidf_abstract_27,w2v_tfidf_abstract_28,w2v_tfidf_abstract_29,w2v_tfidf_abstract_30,w2v_tfidf_abstract_31,w2v_tfidf_abstract_32,w2v_tfidf_abstract_33,w2v_tfidf_abstract_34,w2v_tfidf_abstract_35,w2v_tfidf_abstract_36,w2v_tfidf_abstract_37,w2v_tfidf_abstract_38,w2v_tfidf_abstract_39,...,w2v_tfidf_abstract_160,w2v_tfidf_abstract_161,w2v_tfidf_abstract_162,w2v_tfidf_abstract_163,w2v_tfidf_abstract_164,w2v_tfidf_abstract_165,w2v_tfidf_abstract_166,w2v_tfidf_abstract_167,w2v_tfidf_abstract_168,w2v_tfidf_abstract_169,w2v_tfidf_abstract_170,w2v_tfidf_abstract_171,w2v_tfidf_abstract_172,w2v_tfidf_abstract_173,w2v_tfidf_abstract_174,w2v_tfidf_abstract_175,w2v_tfidf_abstract_176,w2v_tfidf_abstract_177,w2v_tfidf_abstract_178,w2v_tfidf_abstract_179,w2v_tfidf_abstract_180,w2v_tfidf_abstract_181,w2v_tfidf_abstract_182,w2v_tfidf_abstract_183,w2v_tfidf_abstract_184,w2v_tfidf_abstract_185,w2v_tfidf_abstract_186,w2v_tfidf_abstract_187,w2v_tfidf_abstract_188,w2v_tfidf_abstract_189,w2v_tfidf_abstract_190,w2v_tfidf_abstract_191,w2v_tfidf_abstract_192,w2v_tfidf_abstract_193,w2v_tfidf_abstract_194,w2v_tfidf_abstract_195,w2v_tfidf_abstract_196,w2v_tfidf_abstract_197,w2v_tfidf_abstract_198,w2v_tfidf_abstract_199
0,0.013840,-0.228220,0.020195,-0.101649,0.275553,-0.088082,-0.563374,0.336846,-0.283642,-0.698856,0.214058,0.450161,0.224942,0.147445,0.186971,0.470243,0.266813,-0.440381,0.091895,-0.485873,0.337982,0.220681,-0.174911,-0.278267,0.079731,-0.203076,0.072538,0.035059,0.321417,0.136327,0.413391,-0.318785,0.094138,0.119136,-0.156429,0.345598,0.313010,0.222469,-0.089128,-0.385744,...,-0.207123,-0.214017,-0.004177,-0.012512,-0.220373,-0.202233,-0.114127,-0.056878,0.089180,-0.118061,-0.176651,-0.091866,0.147294,-0.169830,-0.431588,0.141046,0.202495,0.180666,-0.136425,-0.473820,0.098303,-0.063823,-0.159296,0.453988,0.091734,0.248894,0.010764,-0.017568,-0.341072,0.211490,0.550860,0.144642,0.425225,0.094566,0.117687,-0.043619,0.119892,-0.247119,-0.052013,0.280306
1,-0.262145,-0.198191,0.194927,-0.059471,0.011627,-0.182048,-0.250211,0.207970,-0.231856,0.033455,0.278028,0.131102,-0.404191,0.110176,0.172630,0.141537,0.001351,0.035858,-0.062008,0.049646,-0.064344,0.191723,-0.146098,0.072273,0.115687,0.153358,0.206181,0.030058,0.250311,-0.073619,0.370482,-0.149637,0.236699,-0.379351,0.001619,-0.104574,0.159792,-0.102476,0.137386,0.192799,...,-0.349179,0.146546,-0.125309,-0.138097,-0.250942,-0.090814,-0.188372,0.011551,-0.092926,0.039653,-0.019086,-0.143241,-0.384244,-0.039037,-0.387538,0.095456,0.148407,-0.316205,-0.177503,-0.370683,-0.374616,-0.034522,-0.088329,0.241431,0.026179,0.349612,-0.204886,-0.010808,-0.434363,0.536534,0.158660,0.076194,0.204102,0.440198,0.094099,-0.257378,0.012963,-0.012675,0.164113,0.032062
2,-0.238870,-0.042947,0.067668,-0.078469,0.094322,-0.070984,-0.328446,0.165365,-0.167163,-0.081546,0.168099,0.075370,-0.071456,0.063036,-0.082335,-0.011103,0.043980,0.188802,-0.067892,0.037408,-0.008209,0.199112,-0.045590,-0.024627,0.005937,0.014488,0.146684,0.082727,0.263194,-0.020310,0.194199,-0.035991,0.187230,-0.058741,0.057122,-0.050502,0.224946,-0.082101,0.076045,0.047409,...,-0.359353,-0.103409,-0.098914,-0.336604,0.045486,-0.043015,-0.067805,0.036391,0.107951,0.041989,0.106160,-0.044372,-0.229888,-0.271248,-0.256426,-0.021295,0.104612,-0.049944,-0.241486,-0.325156,-0.218499,-0.066853,-0.002182,0.066767,0.054802,0.139932,0.016384,-0.048634,-0.259779,0.185943,0.16

In [18]:
abstract_df.to_feather(os.path.join(INPUT_DIR, 'abstract_w2v_tfidf_df.feather'))